In [3]:
!pip install yahoofinancials                                           #Install Lib, More info: https://pypi.org/project/yahoofinancials/
!pip install yfinance

import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from datetime import date
from datetime import datetime, timedelta

##Latest DCF on Zerodha Varsity >> https://zerodha.com/varsity/chapter/discounted-cash-flow-analysis-dcf/
##Ref link-https://towardsdatascience.com/discounted-cash-flow-with-python-f5103921942e >> https://www.youtube.com/watch?v=XCMsdi3CLGA
##         https://medium.com/@polanitzer/building-a-dcf-valuation-in-python-step-by-step-9ba686e0b3a
##         https://pypi.org/project/dcf/ ,https://www.kaggle.com/code/codygordon/discounted-cash-flow-intrinsic-value-model
##         https://einvestingforbeginners.com/dcf-valuation/

"""BELOW DCF MODEL is based on Zerodha Varsity Latest DCF Analysis Module >> https://zerodha.com/varsity/chapter/discounted-cash-flow-analysis-dcf/"""

#The beta of the stock is pretty easy to calculate. I’ve explained it in this chapter "https://zerodha.com/varsity/chapter/hedging-futures/". Refer to section 11.5. 
#I’ll assume the beta of the company .we are modeling as 1.2. As you may know, a beta of 1.2 is high beta. But don’t worry; you can change these numbers anytime since 
#this is an integrated financial model.

##Calculate BETA:
today = datetime.now()
today = today.strftime("%Y-%m-%d")   #Different API used,different datetime formats
last_six_month = datetime.today() - timedelta(days=180)
last_six_month = last_six_month.strftime("%Y-%m-%d")
#Extracting Last 1 year data of NF using yfinance
# index_beta = yf.download('^NSEI', start = last_one_year)
# stock_beta = yf.download('PDMJEPAPER.NS', start = last_one_year)

symbols = ['PDMJEPAPER.NS', '^NSEI']  ##Symbol >> symbols = [stock, index]
beta_df = yf.download(symbols, start = last_six_month)['Adj Close']

# Convert historical stock prices to daily percent change
daily_pct_change = round(beta_df.pct_change(),2)
# print("Daily Pct Change", daily_pct_change)

# Deletes row one containing the NaN
clean_data = daily_pct_change.drop(daily_pct_change.index[0])

from scipy import stats
# Create arrays for x and y variables in the regression model
# Set up the model and define the type of regression
x = np.array(clean_data['PDMJEPAPER.NS']).reshape((-1,1))  #x is stock for which beta needs to be calculated
y = np.array(clean_data['^NSEI'])                          #y is Nifty 50 Index from which beta value for stock is derived using linear regression model
model = LinearRegression().fit(x, y)                       #Linear Regression Model to Calculate β
beta = round(model.coef_[0]*10,2) 
print('Beta = ', beta)  #β

#Another Method to Calculate BETA
# yahoo_financials = YahooFinancials('PDMJEPAPER.NS')
# print("Yahoo Financial BETA:", yahoo_financials.get_beta())

#Another method to Calculat BETA using Variance and Covariance.
"""
cov = np.cov(clean_data['PDMJEPAPER.NS'], clean_data['^NSEI'])
var = np.var(clean_data['^NSEI'])
print(cov,var)
print("BETA:", cov[1,0]/var)
"""

print("All below values in percent '%'' ")

#Risk free rate(rfr) is the current rate of soverign government bonds with has no risk and giving intrest in percent.It will change as per RBI interest rates.
#The data is available for you on RBI’s website. As of today, I’ll take the 10-year bond’s yield as a proxy, which is at 7%
rfr = 7 #Rf
print("rfr", rfr, "%")

#The expected market return(emr) is the standard market expectation and can range between 10% and 12%. Let us go with 12% for now.
emr = 12 #Rm
print("emr:",emr, "%")

#The cost of Equity(coe) is derived from the CAPM(The Capital Asset Pricing mode) formula "Re = Rf + β *( Rf – Rm)" discussed in the previous chapters. It is the risk-free rate plus the 
#difference between the expected market rate and the risk-free rate multiplied by the company’s beta. It is easy if you look at the excel formula and also in python.
#Ref link- https://zerodha.com/varsity/chapter/valuation-part-3-risk-premium-tax-shield/
coe = round(rfr + beta * (emr - rfr),2) #Re = Rf + β *( Rf – Rm)
print("coe:",coe, "%")

#The cost of debt(cod) is the rate at which the company borrows funds—assuming this to be 9%.
cod = 9
print("cod:",cod ,"%")

#The tax rate is 25%. Of course, you can change this to any percentage you think makes sense.For all Corporate Entities, Tax rate is nearly 25%
tax_rate = 25
print("tax_rate:",tax_rate ,"%")

#The target debt-to-equity ratio(de) is assumed to be 50%. While it’s nice to be debt-free, most companies cannot afford to be. They do end up taking debt to fund CAPEX, 
#but a well-run comthe 50% threshold. 
#d/e = Total Equity / (Short term borrowing + company will aim not to cross long term borrowing).
#d/e ratio ca be calculated from Annual Report of Company from BalaceSheet section amd will vary from company to company.
de = 50
print("de:",de, "%")

#The terminal growth rate(tgr) is a super important assumption that we make. The entire DCF model relies heavily on this assumption.
#As discussed in the previous chapter - https://zerodha.com/varsity/chapter/weighted-average-cost-of-capital-and-terminal-growth/, 
#We will assume the terminal growth rate to be close to the long-term inflation number of the country, so between 4 and 5%.
#Check Inflation Rate - https://www.statista.com/statistics/271322/inflation-rate-in-india/
tgr = 5
print("tgr:",tgr, "%")

#The weighted average cost of capital (WACC) is something that we will calculate in excel directly. But I do hope you recollect the discussion we had previously on WACC.
#WACC Ref link- https://zerodha.com/varsity/chapter/weighted-average-cost-of-capital-and-terminal-growth/

#effective cost of debt(ecod) is : Cost of Debt * (1-Tax rate)  
"""IMP >> coe,cod,tax_rate,de,tgr is in percentage, so need to convert these value in percentage >> dividing by 100"""
ecod = round((cod/100 * (1 - tax_rate/100))*100,2)
print("ecod:",ecod, "%")

#The total capital = Debt + Equity = 300 + 300 = 600 Crs (this can be calculated from Annual Report in Balace Sheet section of Company)
#Weight of debt or d/e ratio (de) = 300 / 600 = 50%
#Weight of equity = (1 - weight of debt) = 1 - 50% = 50%
#return expectation of debt holders(ecod) = Cost of Debt *(1-Tax rate) = cod * (1 - tax_rate) = 
#WACC is  = Weight of debt(de) * return expectation of debt holders(ecod) + return expectation of equity holders(coe) * weight of equity
#Hence, the blended rate or WACC is = de * ecod + coe * emr = 50% * 7.5% + 50% * 13%  = 10.25%

"""WACC is implemented in multiple ways as stated above, current version of WACC is used below and giving accurate results during testing.WACC is in percetage terms"""
wacc = ((cod/100) * (1 - (tax_rate/100)) * (de/100) + (coe/100) * (1 - (de/100))) * 100
print("wacc:",wacc, "%") 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
[*********************100%***********************]  2 of 2 completed
Beta =  1.07
All below values in percent '%'' 
rfr 7 %
emr: 12 %
coe: 12.35 %
cod: 9 %
tax_rate: 25 %
de: 50 %
tgr: 5 %
ecod: 6.75 %
wacc: 9.55 %


In [5]:
from yahoofinancials import YahooFinancials

#Calculating DCF for Pudumjee Paper Company. You can get complete info here: https://www.screener.in/company/PDMJEPAPER/
symbol = 'PDMJEPAPER.NS'
yahoo_financials = YahooFinancials(symbol)
cmp = yahoo_financials.get_current_price()

"""
TEST CODE:

# print(yahoo_financials.get_key_statistics_data())
# hi = yahoo_financials.get_financial_stmts('annual', ['income', 'cash', 'balance'])
# income = yahoo_financials.get_financial_stmts('annual', 'income')
# income = income.pop('incomeStatementHistory')
# income = income.pop('INFY.NS')

# # pd.set_option('display.max_rows', 500)
# # pd.set_option('display.max_columns', 500)
# # ##https://stackoverflow.com/questions/73693499/convert-list-of-nested-dictionary-to-pandas-dataframe/73707089#73707089
# income = pd.DataFrame({'date': k, **v} for d in income for k, v in d.items())
# income
# cash = yahoo_financials.get_financial_stmts('annual', 'cash')
# cash = cash.pop('cashflowStatementHistory')
# cash = cash.pop('PDMJEPAPER.NS')
# cash = pd.DataFrame({'date': k, **v} for d in cash for k, v in d.items())

# balance = yahoo_financials.get_financial_stmts('annual', 'balance')
# balance = balance.pop('balanceSheetHistory')
# balance = balance.pop('PDMJEPAPER.NS')
# balance = pd.DataFrame({'date': k, **v} for d in balance for k, v in d.items())
***********************************************************************************************

Getting Key Financial Ratio's using API's
# print(yahoo_financials.get_pe_ratio())
# print(yahoo_financials.get_beta())
# print(yahoo_financials.get_market_cap())
# print(yahoo_financials.get_ebit())
"""

fin_stm_api = yahoo_financials.get_financial_stmts('annual', ['income', 'cash', 'balance'])

date_indexed_dict = {}
for sub_dict in fin_stm_api.values():
    for list_item in sub_dict[symbol]:
        for date, col_dict in list_item.items():
            if date not in date_indexed_dict:
                date_indexed_dict[date] = {}
            # add the columns as keys on each index (date)
            for col, val in col_dict.items():
                date_indexed_dict[date][col] = val

# print(date_indexed_dict)
fin_stm = pd.DataFrame.from_dict(date_indexed_dict, orient='index')
fin_stm.insert(loc=0, column='Symbol', value=symbol)


fin_stm.index.names = ['Date']

"""EBIT is also known as operating income since they both exclude interest expenses and taxes from their calculations."""
fin_stm['EBIT *(1 - Tax Rate'] = fin_stm['ebit'] * (1 - (tax_rate/100))
#https://www.educba.com/change-in-net-working-capital-formula/
# fin_stm['net_working_capital'] = fin_stm['totalCurrentLiabilities'] - fin_stm['otherCurrentAssets']  - fin_stm['inventory']
fin_stm['fcf'] = fin_stm['totalCashFromOperatingActivities'] - (- fin_stm['capitalExpenditures']) 
avg_cf = int(fin_stm['fcf'].mean())

fin_stm['net_working_capital'] = fin_stm['changeToInventory'] + fin_stm['changeToAccountReceivables'] + fin_stm['changeToLiabilities']
fin_stm['FCF'] = fin_stm["EBIT *(1 - Tax Rate"] + fin_stm['depreciation'] - fin_stm['capitalExpenditures'] - fin_stm['net_working_capital']

t_g_v = round(fin_stm['FCF'].tail(1) * (1 + (tgr/100) / ( (wacc-tgr)/100) ) ).values[0]
print("Terminal Growth Value", t_g_v)

lst = [1,2,3,4]                                                     #Count of years, calculating DCF based on last 4 Years of FCF
discount_factor = []
for i in lst:
    dis_factor =  (1 / ( 1 + ( wacc / 100 ) ) )**i
    discount_factor.append(dis_factor)
fin_stm['discount_factor'] = discount_factor
 
fin_stm['pv_of_fcf'] = fin_stm['FCF'] * fin_stm['discount_factor']
#enterprise value (EV) is the total value of a company, defined in terms of its financing. It includes both the current share price (market capitalization) and the cost
# to pay off debt (net debt, or debt minus cash)
pv_of_t_g_v = round(fin_stm['discount_factor'].tail(1) * t_g_v).values[0]
print("Present Value of Terminal Growth Value", pv_of_t_g_v)

ev = round(fin_stm['pv_of_fcf'].sum() + pv_of_t_g_v)
print("Enterprise Value:",ev)

total_debt = - (fin_stm['shortLongTermDebt'].head(1) + fin_stm['longTermDebt'].head(1)).values[0] #Add - sign for debt calculations 
print("Total Debt ₹", total_debt)

equity_value = round(ev + (total_debt) + fin_stm['cash'].head(1)).values[0] 
print("Equity Value:", equity_value)                                 #ev value is not in cr

#share_capital &face_value will be part of DCF Analysis, but it's not using  anywhere during the Analysis.
#Both Parameters will be just for Observation & Analysis purpose while calculating DCF of 100's of Stocks.

share_capital = 10000000                                            #present in balancesheet of Annual Report,No API to get Share Capital Data
face_value = 1                                                      #No API to retrieve Face Value of stock

total_shares = 94950000	                                            #Taking it Manually From Annual Report,Let me know if any API Available to get Total Shares of Any Company.
intrinsic_value = round((equity_value/total_shares),2)              #Intrinsic price(value) of stock in today's date
print("Intrinsic Value ₹",intrinsic_value)                          #in Rs
print("CMP ₹",cmp)                                                  #in Rs

#mos_40 is the Margin of Safety which is considered as 40%, value can change based on individual views.
#MOS 40% means Stocks can be brought as Discount price if MOS 40% is matching or anywhere near to current CMP.
#Margin of Safety provide good protection at downside while buying quality stocks,mostly quality stocks are expensives but there's still opportunity in Markets.
mos_40 = round( intrinsic_value - ( ( intrinsic_value / 10 ) * 4 ) )     
print("MOS 40% ₹" , mos_40)

# fin_stm dataframe is used to Store Data of particular stock during DCF Calculation based on P&L, Balancesheet & CashFlow Statement
pd.set_option('display.width', 1000)
fin_stm 

Terminal Growth Value 1631201918.0
Present Value of Terminal Growth Value 1132552116.0
Enterprise Value: 3636193874
Total Debt ₹ -661894000
Equity Value: 3032998874
Intrinsic Value ₹ 31.94
CMP ₹ 49.35
MOS 40% ₹ 19


,Symbol,researchDevelopment,effectOfAccountingCharges,incomeBeforeTax,minorityInterest,netIncome,sellingGeneralAdministrative,grossProfit,ebit,operatingIncome,...,longTermDebt,inventory,accountsPayable,changeToOperatingActivities,EBIT *(1 - Tax Rate,fcf,net_working_capital,FCF,discount_factor,pv_of_fcf
Date,,,,,,,,,,,,,,,,,,,,,
2022-03-31,PDMJEPAPER.NS,None,None,459976000,None,345320000,627600000,1611521000,578900000,578900000,...,543863000,1218295000,806934000,NaN,434175000.0,-535153000,-367401000,1.621356e+09,0.912825,1.480015e+09
2021-03-31,PDMJEPAPER.NS,None,None,346960000,None,300310000,491611000,1394062000,572786000,572786000,...,180217000,806922000,825262000,NaN,429589500.0,824464000,447692000,1.206775e+08,0.833250,1.005545e+08
2020-03-31,PDMJEPAPER.NS,None,None,410513000,None,272125000,513966000,1462001000,486852000,486852000,...,446421000,919290000,819941000,NaN,365139000.0,447219000,49696000,5.041730e+08,0.760611,3.834798e+08
2019-03-31,PDMJEPAPER.NS,None,None,256207000,None,167424000,519396000,1258077000,342542000,342542000,...,689770000,945735000,828472000,43726000.0,256906500.0,-72771000,-236712000,7.771695e+08,0.694305,5.395929e+08


In [6]:
#Creating Dataframe for DCF Values of all company's. Currently single company DCF is calculatated and added in DCF dataframe
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
dcf = pd.DataFrame([[today,symbol,beta,rfr,emr,coe,cod,tax_rate,de,tgr,ecod,wacc,t_g_v,pv_of_t_g_v,ev,total_debt,equity_value,share_capital,face_value,intrinsic_value,cmp,mos_40]],
                   columns=['Date','Symbol','Beta','RFR %','EMR %',"COE %","COD %","TAX RATE %","DE %","TGR %", "ECOD %","WACC %","TGV", "PV of TGV","Enterprise Value","Total Debt ₹",
                            "Equity Value","Share Capital","Face Value ₹","Intrinsic Value ₹","CMP ₹","MOS 40% ₹"])
dcf

,Date,Symbol,Beta,RFR %,EMR %,COE %,COD %,TAX RATE %,DE %,TGR %,...,TGV,PV of TGV,Enterprise Value,Total Debt ₹,Equity Value,Share Capital,Face Value ₹,Intrinsic Value ₹,CMP ₹,MOS 40% ₹
0,2022-11-13,PDMJEPAPER.NS,1.07,7,12,12.35,9,25,50,5,...,1631201918.00,1132552116.00,3636193874,-661894000,3032998874,10000000,1,31.94,49.35,19
